# Anomalies

In [1]:
import jinja2
import os
import pandas as pd
#from scipy.optimize import minimize
import statsmodels.api as sm

#%pylab inline

c:\users\samth\miniconda3\envs\econ\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
F = pd.read_stata("C:/Data/Thesis/Factors.dta").set_index("date")

In [3]:
models = {
    "CAPM": {"f": ["rm",],
             "regstr": r'R^i=a^i+\beta^iMkt',
             "coefs": ["a", "b"],
             "display_coefs": ["a", "b"]},
    "FF1993": {"f": ["rm", "smb", "hml"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal',
               "coefs": ["a", "b", "s", "v"],
               "display_coefs": ["a", "b", "s", "v"]},
    "C1997": {"f" : ["rm", "smb", "hml", "wml"],
              "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
              "coefs": ["a", "b", "s", "v", "m"],
              "display_coefs": ["a", "v", "m"]},
    "C1997b": {"f" : ["rm", "smb", "HMLm", "wml"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
               "coefs": ["a", "s", "b", "v", "m"],
               "display_coefs": ["a", "v", "m"]},
    "FF2015": {"f" : ["rm", "smb", "hml", "rmw", "cma"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
               "coefs": ["a", "s", "b", "v", "p", "i"],
              "display_coefs": ["a", "v", "p", "i"]},
    "FF2016": {"f" : ["rm", "smb", "hml", "RMWc", "cma"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
               "coefs": ["a", "b", "s", "v", "p", "i"],
              "display_coefs": ["a", "v", "p", "i"]},
    "B2016": {"f" : ["rm", "smb", "HMLm", "wml", "RMWc"],
              "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
              "coefs": ["a", "b", "s", "v", "m", "p"],
              "display_coefs": ["a", "v", "m", "p"]},
}

In [4]:
def tbl_content(LHS, factors, start, end, coef_names, K, j, precision=2, transpose_j=False):

    N = len(LHS.columns)
    A = {name: {"coef": [], "tval": [], "rows": []} for name in coef_names}

    # for each anomaly portfolio
    for lhsi, lhs in enumerate(LHS.columns):

        lhs = LHS[start:end][lhs] - F[start:end]["rf"]  # DataFrame
        rhs = sm.add_constant(F[start:end][factors])    # DataFrame

        # regress the "anomaly" on the factors
        model = sm.OLS(lhs, rhs)
        fit = model.fit()

        # get the coefficients and their t-values
        #coef = [round(b, precision) for b in fit.params]
        #tval = [round(t, precision) for t in fit.tvalues]

        coef = ["{:.2f}".format(b) for b in fit.params]
        tval = ["{:.2f}".format(t) for t in fit.tvalues]

        # for each coefficient
        for namei, name in enumerate(coef_names):  # ordered
            A[name]["coef"].append(coef[namei])
            A[name]["tval"].append(tval[namei])

    for coef in A:

        A[coef]["coef"] = pd.np.array(A[coef]["coef"]).reshape(K, j, j)
        A[coef]["tval"] = pd.np.array(A[coef]["tval"]).reshape(K, j, j)

        if transpose_j:
            for k, x in enumerate(A[coef]["coef"]):
                A[coef]["coef"][k] = A[coef]["coef"][k].transpose()
                A[coef]["tval"][k] = A[coef]["tval"][k].transpose()

        A[coef]["coef"] = A[coef]["coef"].tolist()
        A[coef]["tval"] = A[coef]["tval"].tolist()

    for coef in A:
        for row in range(j):
            A[coef]["rows"].append([])
            for k in range(K):
                A[coef]["rows"][row].extend(A[coef]["coef"][k][row])
                A[coef]["rows"][row].extend(A[coef]["tval"][k][row])
    return A

In [5]:
path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

## 2x4x4

In [6]:
template = env.get_template("2x4x4_template.txt")

### Size-OP-Inv

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"), 2, 4
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "32_Size_OP_Inv"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "OP", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

### Size-BM-Inv

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"), 2, 4
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "32_Size_BM_Inv"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "BM", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

### Size-BM-Prior

In [9]:
a, K, j = pd.read_csv("C:/Data/Thesis/32_Portfolios_ME_BM_PR.csv"), 2, 4
a = a.iloc[:642, 1:]
a.index = F.index

a = a.fillna(-99.99)

fname_prefix = "32_Size_BM_Prior"

start, end = "1963-07", "2016-12"

In [10]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "Prior", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

C1997b
C1997
FF1993
B2016
FF2015
FF2016
CAPM


## 5x5

In [ ]:
template = env.get_template("5x5_template.txt")

### Size-Inv

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "25_Size_Inv"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "Inv", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

### Size-Var

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "25_Size_Var"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "Var", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

### Size-Prior

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "25_Size_Prior"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "Prior", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))

### Size-Beta

In [ ]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

fname_prefix = "25_Size_Beta"

start, end = "1963-07", "2016-12"

In [ ]:
for m in models:
    print(m)
    A = tbl_content(a, models[m]["f"], start, end, models[m]["coefs"], K, j, 2, True)
    context = {
        "A": A, "K": K, "j":j, "coef_names": models[m]["display_coefs"],
        "var_name": "Beta", "title": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(
            fname_prefix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_prefix, m)
    }

    fname = "{}_{}_{}_{}.tex".format(fname_prefix, m, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))